### Model API
Lets see if the LLM can provide us with code that can deploy the model using the library FastAPI

**Prompt:** I have an xgb model saved in my local directory as a pickle file. Can you deploy the model using the library FastAPI?

**Result:**

In [1]:
import pickle

with open('xgb_model.pkl', 'rb') as file:
   model = pickle.load(file)

In [6]:
#Intervened - added my own features
from fastapi import FastAPI
from pydantic import BaseModel, Field #added Field for my features

class Data(BaseModel):
    tenure	:	int
    MonthlyCharges	:	int
    TotalCharges	:	float
    TotalServices	:	float
    gender_Male	:	int
    InternetService_Fiber_optic	:	int = Field(alias='InternetService_Fiber optic')
    InternetService_No	:	int
    Contract_One_year	:	int = Field(alias='Contract_One year')
    Contract_Two_year	:	int = Field(alias='Contract_Two year')
    PaymentMethod_Credit_card_automatic	:	int = Field(alias='PaymentMethod_Credit card (automatic)')
    PaymentMethod_Electronic_check	:	int = Field(alias='PaymentMethod_Electronic check')
    PaymentMethod_Mailed_check	:	int = Field(alias='PaymentMethod_Mailed check')
    SeniorCitizen	:	int
    Partner	:	int
    Dependents	:	int
    PhoneService	:	int
    MultipleLines	:	int
    OnlineSecurity	:	int
    OnlineBackup	:	int
    DeviceProtection	:	int
    TechSupport	:	int
    StreamingTV	:	int
    StreamingMovies	:	int
    PaperlessBilling	:	int

   # Add more features as needed

app = FastAPI()

In [7]:
#Intervened - added my own features
@app.post('/predict')
def predict(data: Data):
   # Prepare data for prediction
   data_to_predict = [[data.tenure, 	data.MonthlyCharges, 	data.TotalCharges,
                       data.TotalServices, 	data.gender_Male, 	data.InternetService_Fiber_optic,
                       data.InternetService_No, 	data.Contract_One_year, 	data.Contract_Two_year,
                       data.PaymentMethod_Credit_card_automatic, 	data.PaymentMethod_Electronic_check,
                       data.PaymentMethod_Mailed_check, 	data.SeniorCitizen, 	data.Partner,
                       data.Dependents, 	data.PhoneService, 	data.MultipleLines,
                       data.OnlineSecurity, 	data.OnlineBackup, 	data.DeviceProtection,
                       data.TechSupport, 	data.StreamingTV, 	data.StreamingMovies, 	data.PaperlessBilling,
                       ]] # Add more features as needed

   # Predict the class
   prediction = model.predict(data_to_predict)

   # Return the result
   return {'prediction': prediction[0]}

In [ ]:
#added this later
if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=80)

In [ ]:
#To be run in console CLI
uvicorn main:app --reload

The code is mostly legit, its missing the uvicorn run method that maps host and port. However the API cannot be launched from here since its a jupyter notebook. It'd need to be in a py file. 

The LLM commentary even told us what we had to adjust to suit our models features.

Now what if we wished to dockerise this API so that we could easily deploy it on a cloud service?

**Prompt:** I want to deploy this with Docker. Can you write a docker file that would accomplish this?

**Result:**

In [ ]:
# Use an official Python runtime as a parent image
FROM python:3.8-slim-buster

# Set work directory
WORKDIR /app

# Install dependencies
COPY requirements.txt /app/
RUN pip install --no-cache-dir -r requirements.txt

# Copy the current directory contents into the container at /app
COPY . /app/

# Make port 80 available to the world outside this container
EXPOSE 80

# Run the application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]

In [ ]:
# Build & run (CLI)
docker build -t my-fastapi-app .
docker run -p 80:80 my-fastapi-app

Well, that would work. The environment variables are not necessary though (removed in this notebook). A few edits to conform to the repo might be needed too. It's not too dissimilar to what FastAPI recommends: https://fastapi.tiangolo.com/deployment/docker/